In [ ]:
# Narejeno po članku
# https://datachef.co/blog/paraphrasing-with-gpt2/
# Za to uporablja datoteko PrevodiClarin.xlsx, ki je tudi na Git-u
# https://github.com/UL-FRI-NLP-Course-2022-23/nlp-course-parafrizerji/blob/main/database/PrevodiClarin.xlsx

In [1]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.2 MB/s eta 0:00:00


In [2]:
# importi
from google.colab import drive
import pandas as pd

In [3]:
drive.mount('/content/gdrive/', force_remount=True)
skupna = pd.read_excel("/content/gdrive/MyDrive/NLP/PrevodiClarin.xlsx")
#print(skupna)

Mounted at /content/gdrive/


In [ ]:
# naredi novo kolono v datasetu
skupna["parafraziranje"] = '<s>'+skupna['parafraza']+'</s>'+'>>>>'+'<p>'+skupna['original']+'</p>'

In [ ]:
# shranim novo kolono v svojo datoteko
skupna['parafraziranje'] = skupna.parafraziranje.to_csv('parafraziranje.txt',sep='\n',index=False)

In [ ]:
from transformers import (
    AutoModelWithLMHead, 
    AutoConfig,
    Trainer,
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    TrainingArguments)
import os

In [ ]:
def modelTrainer(text_path, epochs, modell='cjvt/gpt-sl-base', batch_size=8):

  print(modell)

  model = AutoModelWithLMHead.from_pretrained(modell)	
  #print("a")
  tokenizer = AutoTokenizer.from_pretrained('cjvt/gpt-sl-base')
  #tokenizer = AutoTokenizer.from_pretrained(modell)
  #print("b")
  data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
  #print("c")
  train_dataset = TextDataset(
      tokenizer=tokenizer, 
      file_path = text_path, 
      block_size=256
  )
  print(type(train_dataset))
  trainig_args = TrainingArguments(
      output_dir="/content/gdrive/MyDrive/NLP/GPTmodel/",
      num_train_epochs = epochs,
      per_device_train_batch_size = batch_size,
      warmup_steps = 500,
      save_steps = 2000,
      logging_steps = 10,
  )
  trainer = Trainer(
		model=model,
		args = trainig_args,
		data_collator = data_collator,
		train_dataset = train_dataset,
    #cache_enabled = False,
		#prediction_loss_only = True
  )

  trainer.train()
  trainer.save_model()

In [ ]:
modelTrainer("parafraziranje.txt", epochs=30, batch_size=8, modell="/content/gdrive/MyDrive/NLP/GPTmodel/")

/content/gdrive/MyDrive/NLP/GPTmodel/


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (364691 > 1024). Running this sequence through the model will result in indexing errors


<class 'transformers.data.datasets.language_modeling.TextDataset'>


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,2.062900
20,2.093500
30,2.220200
40,2.049900
50,2.041000
60,1.883200
70,1.764500
80,1.829500
90,1.693300
100,1.761800


In [10]:
from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("/content/gdrive/MyDrive/NLP/GPTmodel/")
tokenizer = AutoTokenizer.from_pretrained("cjvt/gpt-sl-base")
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=100)


stavkiZaPara = []
stavkiZaPara.append("Še nobeni vladi ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.")
stavkiZaPara.append("Bila so čisto navadna torkova večerna poročila na POP TV, torej oddaja 24ur, in je Darja Zgonc čisto mirno, kot da napoveduje, da vreme pač bo, izrekla naslednjo napoved prispevka.")
stavkiZaPara.append("Življenje samohranilke z dvema otrokoma se vrti nevarno blizu roba.")
stavkiZaPara.append("Janez Vogrinc je dekan Pedagoške fakultete v Ljubljani. ")
stavkiZaPara.append("Je tudi vodja delovne skupine za pripravo novega nacionalnega programa vzgoje in izobraževanja za obdobje 2023–2033, ki jo je oblikovalo resorno ministrstvo.")
stavkiZaPara.append("Glasbeni megahit, ki ga je še danes mogoče slišati na številnih plesiščih (ali pa vsaj po radijskih valovih), je opeval tradicionalno moške lastnosti in njihove adute.")
stavkiZaPara.append("Upravno sodišče je sredi letošnjega januarja izdalo odločbo, v kateri je ugodilo tožbi kmeta, ki mu je bilo razlaščeno 17.785 kvadratnih metrov kmetijskega zemljišča v korist Občine Hoče - Slivnica.")
stavkiZaPara.append("V okviru 14. Dneva za spremembe Slovenska filantropija po vsej Sloveniji organizira akcije, ki spodbujajo državljane k aktivnemu odzivu na potrebe svoje skupnosti.")
stavkiZaPara.append("Slovenijo je zajela protestniška inflacija.")
stavkiZaPara.append("Protestov je čedalje več, njihova vrednost pa je čedalje manjša.")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [5]:
def paraphrase(input_sentence):
  return generator('<s>'+input_sentence+'</s>>>>><p>')

In [6]:
# Izpis rezultatov
for i in stavkiZaPara:
  print(paraphrase(i))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '<s>Še nobeni vladi ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.</s>>>>><p>Šeniji ni uspelo nikakor uzreti dopolnilnega zdravstvenega zavarovanja, kar v resnici sploh ni zavarovanje.</p>\n<s>Uvedli ste tudi brezplačne prigrizke za srednješolce in brezplačne vrtce za drugega otroka.</s>>>>><p>Uvedli pa ste tudi brezplačne malice za srednješolce in brezplačne vrtce za drugega otroka.</p>\n<s>Pohvaliti velja preprosto brskanj'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '<s>Bila so čisto navadna torkova večerna poročila na POP TV, torej oddaja 24ur, in je Darja Zgonc čisto mirno, kot da napoveduje, da vreme pač bo, izrekla naslednjo napoved prispevka.</s>>>>><p>Bila so povsem navadna litij-ionska obarvana večernaporočila na Pop TV, torej oddaja 24ur, in je Darja Zgonc (Kristina, Marijina hči in Mijina stara teta) povsem mirno, kot da bi popeljala gledalce v počitnice.</p>\n<s>Nova pravila pomenijo, da se letos pričakuje več stečajev in likvidacij.</s>>>>><p>'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

[{'generated_text': '<s>Upravno sodišče je sredi letošnjega januarja izdalo odločbo, v kateri je ugodilo tožbi kmeta, ki mu je bilo razlaščeno 17.785 kvadratnih metrov kmetijskega zemljišča v korist Občine Hoče - Slivnica.</s>>>>><p>Ugodnjo odločbo je sodišča izdalo sredi letošnjega januarja, in sicer v delu koptov, ki jih je tamkajšnje sodišče denacionalizaciji primerno odškodnino.</p>\n<s>Sama pesem je ena tistih, ki jih pozabiš takoj, ko je končana.</s>>>>><p>Sama pesem je ena izmed tistih, ki jih takoj pozabiš.</p>\n<s>'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '<s>V okviru 14. Dneva za spremembe Slovenska filantropija po vsej Sloveniji organizira akcije, ki spodbujajo državljane k aktivnemu odzivu na potrebe svoje skupnosti.</s>>>>><p>V okviru programa 14._ob 20.000 spremembah v skladu s pogodbo o spremembah slovenske prejela Konvencija o oblikovanju pozitivnega odgovora na potrebe svoje skupnosti.</p>\n<s>V četrtek nas je vse poklicala simpatična Pobinovka Urška Pirš.</s>>>>><p>V četrtek nas je vsa iz tira poklicala simpatična Pobinovka Urška Pir'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '<s>Slovenijo je zajela protestniška inflacija.</s>>>>><p>Slovenijo je objela protestniška inflacija.</p>\n<s>mislim, da tega dogodka ni organizirala administracija študentskega doma.</s>>>>><p>Mislim, da ni uprava študentskih domov organizirala prireditev.</p>\n<s>Španec je bil vedno tisti, ki je narekoval tempo, in izkazalo se je, da je Kasparov nadzoroval svoj položaj.</s>'}]
[{'generated_text': '<s>Protestov je čedalje več, njihova vrednost pa je čedalje manjša.</s>>>>><p>Protestu so se vedno pogosteje pridružile tudi druge stranke, njihova vrednost je čedalje večja.</p>\n<s>Ne moremo se izogniti zapletom, ki jih ima Devica.</s>>>>><p>Devica Zapletu se nikakor ne boste mogli izogniti.</p>\n<s>To je tudi ena od slabosti, ki jih skuša novi zakon popraviti.</s>>>'}]


In [15]:
def clean_paraphrased(input_sentence):
  p = generator('<s>'+input_sentence+'</s>>>>><p>')
  return p[0]['generated_text'].split('</s>>>>><p>')[1]#.split('</p>')


In [16]:
stevec=0
for i in stavkiZaPara:
  print(stavkiZaPara[stevec])
  print(clean_paraphrased(i))
  print()
  stevec += 1

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Še nobeni vladi ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Šeniji ni uspelo nikakor uzreti dopolnilnega zdravstvenega zavarovanja, kar v resnici sploh ni zavarovanje.</p>
<s>Uvedli ste tudi brezplačne prigrizke za srednješolce in brezplačne vrtce za drugega otroka.

Bila so čisto navadna torkova večerna poročila na POP TV, torej oddaja 24ur, in je Darja Zgonc čisto mirno, kot da napoveduje, da vreme pač bo, izrekla naslednjo napoved prispevka.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Ugodnjo odločbo je sodišča izdalo sredi letošnjega januarja, in sicer v delu koptov, ki jih je tamkajšnje sodišče denacionalizaciji primerno odškodnino.</p>
<s>Sama pesem je ena tistih, ki jih pozabiš takoj, ko je končana.

V okviru 14. Dneva za spremembe Slovenska filantropija po vsej Sloveniji organizira akcije, ki spodbujajo državljane k aktivnemu odzivu na potrebe svoje skupnosti.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


V okviru programa 14._ob 20.000 spremembah v skladu s pogodbo o spremembah slovenske prejela Konvencija o oblikovanju pozitivnega odgovora na potrebe svoje skupnosti.</p>
<s>V četrtek nas je vse poklicala simpatična Pobinovka Urška Pirš.

Slovenijo je zajela protestniška inflacija.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Slovenijo je objela protestniška inflacija.</p>
<s>mislim, da tega dogodka ni organizirala administracija študentskega doma.

Protestov je čedalje več, njihova vrednost pa je čedalje manjša.
Protestu so se vedno pogosteje pridružile tudi druge stranke, njihova vrednost je čedalje večja.</p>
<s>Ne moremo se izogniti zapletom, ki jih ima Devica.



In [ ]:
# Zbriše cache stavkov. Če ne poženeš tega ukaza preprosto vzame iste stavke za nov trening.
!rm -rf cac*